In [1]:
!pip install bitsandbytes accelerate
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 624.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 32.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


**Bitsandbytes**: A lightweight CUDA library for quantized matrix multiplication, useful for faster and more memory-efficient computations.

**Accelerate**: A library by Hugging Face to accelerate training and inference on multiple devices, including CPUs, GPUs, and TPUs.

**Device Mapping for Model Layers**
When dealing with large models, distributing the layers across multiple devices can help in managing memory and computational resources more efficiently. The device_maps variable is used to specify which layers of the model are assigned to which device.

In [2]:
device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_token")

In [ ]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)

By using torch.backends.cuda.enable_mem_efficient_sdp(False), you can disable memory-efficient SDP in PyTorch, which might be necessary for certain models or debugging purposes. This configuration can help achieve better performance or consistency across different hardware setups.

In [4]:
from transformers import (AutoTokenizer, AutoModelForCausalLM, 
                          BitsAndBytesConfig, AutoConfig)

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model_name = "google/gemma-2-9b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token =secret_value_0) 


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [5]:
device= {layer:gpu_mem for (layer,gpu_mem) in device_maps}

In [10]:
 
config = AutoConfig.from_pretrained(model_name,token=secret_value_0)
config.gradient_checkpointing = True

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

In [11]:
config

Gemma2Config {
  "_name_or_path": "google/gemma-2-9b-it",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "gradient_checkpointing": true,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 16,
  "num_hidden_layers": 42,
  "num_key_value_heads": 8,
  "pad_token_id": 0,
  "query_pre_attn_scalar": 224,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "sliding_window_size": 4096,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.42.3",
  "use_cache": true,
  "vocab_size": 256000
}

In [12]:

model = AutoModelForCausalLM.from_pretrained(model_name ,torch_dtype="auto",quantization_config=quantization_config,token =secret_value_0,
                                             device_map="auto",trust_remote_code=True,config=config)

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [15]:
input_text = "correct this sentense: He does eat meat everyday"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids,max_length=150)
print(tokenizer.decode(outputs[0]))

2024-06-29 02:28:22.551394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 02:28:22.551496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 02:28:22.825696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


<bos>correct this sentense: He does eat meat everyday.

The corrected sentence is: **He eats meat every day.**


Here's why:

* **Subject-Verb Agreement:**  "He" is singular, so the verb needs to be "eats" (singular present tense) instead of "does eat" (present tense with auxiliary verb).
* **Word Order:**  In English, the typical word order for this type of sentence is subject-verb-adverbial.  "Every day" is an adverbial phrase modifying "eats". 



Let me know if you have any other sentences you'd like help with!<end_of_turn>
<eos>
